In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn import datasets
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.datasets import load_digits
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from scipy.stats import sem
from numpy import linalg as LA
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

## Calculating the margin

In [2]:
def compute_margin(X, y, w, b):
    margin = []
    for i in range(y.shape[0]):
        margin.append((y[i]*(np.dot(X[i,:], w)+b))/sqrt(sum(w**2))) 
    if min(margin) >= 0:
        return min(margin)
    return 0

## ELM-IMA with W_init = [0, ..., 0] 

In [3]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin
class ELM_IMA(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        norm_w = sqrt(sum(w**2))
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * norm_w - self.lambda_param * alpha[i]):
                    if norm_w != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/norm_w
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w * lambda_t + self.eta * y[i] * X[i,:]
                    norm_w = sqrt(sum(w**2))
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = np.zeros(self.w_elm.shape[0])
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = 0#compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = sqrt(sum(self.w**2))
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
        return t, updates, iterations, margin

    # Function that manage the training of IMA ELM
    def fit(self, X, y):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        self.H = np.tanh(np.dot(X, self.Z))
        w = np.dot(np.linalg.pinv(self.H), y)  
        self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations, margin = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations, margin
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        else:
            y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## ELM-IMA with W_init = $\beta$

In [4]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin
class ELM_IMA_b(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        norm_w = sqrt(sum(w**2))
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * norm_w - self.lambda_param * alpha[i]):
                    if norm_w != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/norm_w
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w * lambda_t + self.eta * y[i] * X[i,:]
                    norm_w = sqrt(sum(w**2))
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = deepcopy(self.w_elm)
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)            
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = sqrt(sum(self.w**2))
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
        return t, updates, iterations, margin

    # Function that manage the training of IMA ELM   
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        w = np.dot(np.linalg.pinv(self.H), y)  
        self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations, margin = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations, margin
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        else:
            y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## Commom ELM

In [5]:
class ELM(BaseEstimator, ClassifierMixin):
         
    def __init__(self, n_neurons):
        self.n_neurons = n_neurons
    
    def fit(self, X, y):
        # Adding polarization term 
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        n = X_new.shape[1]
        self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        H = np.tanh(np.dot(X_new, self.Z))
        self.w = np.dot(np.linalg.pinv(H), y)           
        return self.w, H, self.Z
            
    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        y_predicted = np.sign(np.dot(H, self.w))
        y_predicted[y_predicted==0]=1
        return y_predicted

## Function to get Results

In [6]:
def grid_C(X, y):
    # GridSearch for C parameter of SVM:
    parameters = {'C':np.logspace(-3, 2, 50)}
    clf = svm.SVC(kernel="rbf", gamma='scale')
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=10, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['C']

In [7]:
def grid_l(X, y, p):
    # GridSearch for lambda and learning rate of IMA ELM
    parameters = {'lambda_param':np.linspace(0.01, 10, 50)}
    clf = ELM_IMA(n_neurons=p, delta_margin=10^-3, IMA_iterations=10, max_updates=100000)
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=5, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['lambda_param']
    

In [8]:
def results(X, y, n_splits, p, eta, IMA_iterations, lambda_param, C):      
    print(f'Parameters: p={p}, eta={eta}, lambda={lambda_param}, C={C}')
    # Stratified k fold cross validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=72)
    i=0
    
    train_accuracy_IM_ELM = np.zeros(n_splits)
    test_accuracy_IM_ELM = np.zeros(n_splits)
    margin_IM_ELM = np.zeros(n_splits)
    updates = np.zeros(n_splits)
    iterations_FMP = np.zeros(n_splits) 
    iterations_IMA = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_b = np.zeros(n_splits)
    test_accuracy_IM_ELM_b = np.zeros(n_splits)
    margin_IM_ELM_b = np.zeros(n_splits)
    updates_b = np.zeros(n_splits)
    iterations_FMP_b = np.zeros(n_splits) 
    iterations_IMA_b = np.zeros(n_splits)
          
    train_accuracy_ELM = np.zeros(n_splits)
    test_accuracy_ELM = np.zeros(n_splits)
    margin_ELM = np.zeros(n_splits)
    
    train_accuracy_SVM = np.zeros(n_splits)
    test_accuracy_SVM = np.zeros(n_splits)
    margin_SVM = np.zeros(n_splits)

    margins=[]
    margins_b = []
        
    for train_index, test_index in kf.split(X, y):
        X_train = X[train_index,:]
        X_test = X[test_index,:]
        y_train = y[train_index]
        y_test = y[test_index]
               
        # ELM-IMA
        clf = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3, IMA_iterations=IMA_iterations, max_updates=10000)
        iterations_IMA[i], updates[i], iterations_FMP[i], margin = clf.fit(X_train, y_train)
        margins.append(margin)
        y_hat=clf.predict(X_test, use_IMA_w = True)
        y_hat_train=clf.predict(X_train, use_IMA_w = True)
        margin_IM_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w, clf.b)
        train_accuracy_IM_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM[i] = accuracy_score(y_test, y_hat)
        Z = clf.Z
                
        # ELM
        y_hat=clf.predict(X_test, use_IMA_w = False)
        y_hat_train=clf.predict(X_train, use_IMA_w = False)
        margin_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w_elm, 0)
        train_accuracy_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_ELM[i] = accuracy_score(y_test, y_hat)
                
         # ELM-IMA (beta)
        clf = ELM_IMA_b(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3, IMA_iterations=IMA_iterations, max_updates=10000)
        iterations_IMA_b[i], updates_b[i], iterations_FMP_b[i], margin_b = clf.fit(X_train, y_train, Z)
        margins_b.append(margin_b)
        y_hat=clf.predict(X_test, use_IMA_w = True)
        y_hat_train=clf.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_b[i] = compute_margin(clf.H[:,:], y_train, clf.w, clf.b)
        train_accuracy_IM_ELM_b[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_b[i] = accuracy_score(y_test, y_hat)
                     
        # SVC Classifier
        clf = svm.SVC(kernel="rbf", C=C, gamma='scale')
        clf.fit(X_train, y_train)
        y_hat=clf.predict(X_test)
        y_hat_train=clf.predict(X_train)
        train_accuracy_SVM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_SVM[i] = accuracy_score(y_test, y_hat)
        i+=1
        
    print("********* Results ELM-IMA **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA.std()))
    print("Updates: " + '{:.4f}'.format(updates.mean())+ "+/-" + '{:.4f}'.format(updates.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM.std()))
    
    print("********* Results ELM-IMA Beta **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_b.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_b.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_b.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_b.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_b.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_b.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_b.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_b.std()))
    print("Updates: " + '{:.4f}'.format(updates_b.mean())+ "+/-" + '{:.4f}'.format(updates_b.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_b.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_b.std()))
       
    print("********* Results ELM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_ELM.std()))
    print("Margin: " + '{:.9f}'.format(margin_ELM.mean())+ "+/-" + '{:.9f}'.format(margin_ELM.std()))
    
    print("********* Results SVM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_SVM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_SVM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_SVM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_SVM.std()))
    print("Margin: " + '{:.6f}'.format(margin_SVM.mean())+ "+/-" + '{:.6f}'.format(margin_SVM.std()))
    return margins

In [9]:
def run(X,y):
    # Normalizing data:
    X = preprocessing.normalize(X, axis=0)
    C = grid_C(X, y)
    n = len(X)
    if n>1000:
        n=1000
    for p in [n, int(n/2), int(n/3)]:
        l = grid_l(X, y, p=p)
        print(f"Experimento com {p} neurônios:" )
        m = results(X, y, n_splits=10, p=p, eta=0.1, IMA_iterations=20, lambda_param=l, C=C)
        print("\n\n")

In [10]:
iris = datasets.load_iris()
X = iris.data
# setosa - 0, versicolor - 1, virginica - 2  
y = iris.target 
# O problema agora possui apenas as classes y=-1 e y=1
y[y>0] = 1
y[y==0] = -1
run(X, y)
#plot_margin_evolution(m)

Experimento com 150 neurônios:
Parameters: p=150, eta=0.1, lambda=0.01, C=0.021209508879201904
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 1210.7000+/-1379.3446
Iterations IMA: 20.0000+/-0.0000
Updates: 5722.8000+/-4876.7584
Margin: 0.073668463+/-0.004967061
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 484.7000+/-254.9388
Iterations IMA: 20.0000+/-0.0000
Updates: 2096.6000+/-928.8844
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9800+/-0.0427
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.000000+/-0.000000



Experimento com 75 neurônios:
Parameters: p=75, eta=0.1, lambda=0.01, C=0.021209508879201904
********* Results ELM-IMA **************
Acc train: 0.9963+/-0.0111
Acc test: 1.0000+/-0.0000
Iterations: 943.200

In [11]:
synthetic_dataset = pd.read_csv('data/synthetic_dataset/synthetic_control.data', sep="\s+",  header=None, engine='python')
X = synthetic_dataset.to_numpy()
y = np.concatenate((np.ones(100), np.ones(200)*-1, np.ones(100), np.ones(100)*-1,np.ones(100)))
run(X, y)

Experimento com 600 neurônios:
Parameters: p=600, eta=0.1, lambda=0.01, C=0.04291934260128778
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9983+/-0.0050
Iterations: 434.8000+/-248.5489
Iterations IMA: 20.0000+/-0.0000
Updates: 4390.9000+/-2033.9326
Margin: 0.030336831+/-0.007432203
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9817+/-0.0263
Iterations: 293.0000+/-164.3076
Iterations IMA: 20.0000+/-0.0000
Updates: 2658.4000+/-989.9552
Margin: 0.000043240+/-0.000008456
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7517+/-0.0825
Margin: 0.000024857+/-0.000002605
********* Results SVM **************
Acc train: 0.9996+/-0.0011
Acc test: 1.0000+/-0.0000
Margin: 0.000000+/-0.000000



Experimento com 300 neurônios:
Parameters: p=300, eta=0.1, lambda=0.01, C=0.04291934260128778
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9983+/-0.0050
Iterations: 443.6000+

In [12]:
robot_dataset = pd.read_csv('data/robot/lp4_data.csv', delimiter =',')
X = robot_dataset.to_numpy().reshape([117,90])
y = np.concatenate((np.ones(24), np.ones(117-24)*-1))
run(X, y)

Experimento com 117 neurônios:
Parameters: p=117, eta=0.1, lambda=2.2526530612244895, C=19.306977288832496
********* Results ELM-IMA **************
Acc train: 0.8337+/-0.0588
Acc test: 0.7871+/-0.0529
Iterations: 379.7000+/-165.7661
Iterations IMA: 11.5000+/-8.5235
Updates: 7033.4000+/-4148.8974
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5818+/-0.0834
Iterations: 97.6000+/-19.0589
Iterations IMA: 20.0000+/-0.0000
Updates: 496.1000+/-149.1184
Margin: 0.001109329+/-0.000214479
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5561+/-0.0685
Margin: 0.001237265+/-0.000243345
********* Results SVM **************
Acc train: 0.9516+/-0.0100
Acc test: 0.9326+/-0.0503
Margin: 0.000000+/-0.000000



Experimento com 58 neurônios:
Parameters: p=58, eta=0.1, lambda=3.883673469387755, C=19.306977288832496
********* Results ELM-IMA **************
Acc train: 0.7939+/-0.0041
Acc test: 0.7871+/-0.0375
I

In [13]:
df = pd.read_csv('data/Mushroom/agaricus-lepiota.data', delimiter =',', header=None)
df = df.replace("?", np.nan) 
df = df.dropna() 
y = df[0].to_numpy()
X = df.drop([0], axis='columns')
X = pd.get_dummies(X).to_numpy()
y[np.where(y=='e')] = -1
y[np.where(y=='p')] = 1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=4.0875510204081635, C=0.04291934260128778
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 62.9000+/-7.7389
Iterations IMA: 20.0000+/-0.0000
Updates: 1357.7000+/-397.6848
Margin: 0.049897723+/-0.022380742
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 60.0000+/-4.7117
Iterations IMA: 20.0000+/-0.0000
Updates: 1504.2000+/-473.8898
Margin: 0.065234755+/-0.007324467
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.058185186+/-0.002137417
********* Results SVM **************
Acc train: 0.9990+/-0.0005
Acc test: 0.9972+/-0.0016
Margin: 0.000000+/-0.000000



Experimento com 500 neurônios:
Parameters: p=500, eta=0.1, lambda=4.291428571428571, C=0.04291934260128778
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0001
Acc test: 0.9998+/-0.0005

In [14]:
ionosphere_dataset = pd.read_csv('data/Ionosphere/ionosphere.data', names=list(range(0,35)), sep=',')
y = ionosphere_dataset[34].to_numpy()
X = ionosphere_dataset.drop([34], axis='columns').to_numpy()
y[np.where(y=='g')] = 1
y[np.where(y=='b')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 351 neurônios:
Parameters: p=351, eta=0.1, lambda=0.01, C=1.151395399326448
********* Results ELM-IMA **************
Acc train: 0.3590+/-0.0015
Acc test: 0.3590+/-0.0135
Iterations: 207.1000+/-18.9021
Iterations IMA: 1.0000+/-0.0000
Updates: 10029.7000+/-15.9377
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6951+/-0.0860
Iterations: 200.5000+/-13.2382
Iterations IMA: 1.0000+/-0.0000
Updates: 10024.2000+/-16.9163
Margin: 0.000053103+/-0.000014687
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6951+/-0.0860
Margin: 0.000053103+/-0.000014687
********* Results SVM **************
Acc train: 0.9702+/-0.0043
Acc test: 0.9429+/-0.0461
Margin: 0.000000+/-0.000000



Experimento com 175 neurônios:
Parameters: p=175, eta=0.1, lambda=0.01, C=1.151395399326448
********* Results ELM-IMA **************
Acc train: 0.3590+/-0.0015
Acc test: 0.3590+/-0.0135
Iterations: 209.0000+/-16.0187

In [15]:
# read in banknote authentication set
banknotes = pd.read_csv('data/banknote/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])
# convert to array
X = banknotes[['variance', 'skewness', 'curtosis', 'entropy']].to_numpy()
y = banknotes[['class']].to_numpy()
y[np.where(y==0)] = -1
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=0.01, C=0.5689866029018299
********* Results ELM-IMA **************
Acc train: 0.5554+/-0.0001
Acc test: 0.5554+/-0.0013
Iterations: 193.0000+/-32.5423
Iterations IMA: 1.0000+/-0.0000
Updates: 10035.4000+/-19.2156
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9985+/-0.0029
Iterations: 468.7000+/-387.8734
Iterations IMA: 12.6000+/-8.1756
Updates: 9496.1000+/-7382.1771
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9993+/-0.0022
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.000000+/-0.000000



Experimento com 500 neurônios:
Parameters: p=500, eta=0.1, lambda=0.21387755102040817, C=0.5689866029018299
********* Results ELM-IMA **************
Acc train: 0.9864+/-0.0045
Acc test: 0.9891+/-0.0114
Iteratio

In [16]:
wine_dataset = pd.read_csv('data/wine/wine.data', names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline'])
# convert to array
y = wine_dataset[['Class']].to_numpy()
X = wine_dataset.drop("Class",axis='columns').to_numpy()
y[np.where(y==3)] = 1
y[np.where(y==2)] = -1
run(X, y)

Experimento com 178 neurônios:
Parameters: p=178, eta=0.1, lambda=0.01, C=0.9102981779915218
********* Results ELM-IMA **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9660+/-0.0456
Iterations: 838.6000+/-567.5090
Iterations IMA: 20.0000+/-0.0000
Updates: 7356.2000+/-4396.6960
Margin: 0.004436531+/-0.002707674
********* Results ELM-IMA Beta **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9036+/-0.0536
Iterations: 481.2000+/-313.0463
Iterations IMA: 20.0000+/-0.0000
Updates: 4670.1000+/-2879.5398
Margin: 0.000002650+/-0.000000645
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7516+/-0.1011
Margin: 0.000002595+/-0.000000535
********* Results SVM **************
Acc train: 0.9988+/-0.0025
Acc test: 0.9778+/-0.0369
Margin: 0.000000+/-0.000000



Experimento com 89 neurônios:
Parameters: p=89, eta=0.1, lambda=0.6216326530612245, C=0.9102981779915218
********* Results ELM-IMA **************
Acc train: 0.9588+/-0.0258
Acc test: 0.9441+/-0.0556
Iterations

In [17]:
wdbc_dataset = pd.read_csv('data/WDBC/wdbc.data', names=list(range(0,32)))
# convert to array
y = wdbc_dataset[1].to_numpy()
X = wdbc_dataset.drop([0, 1],axis='columns').to_numpy()
y[np.where(y=='B')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 569 neurônios:
Parameters: p=569, eta=0.1, lambda=1.233265306122449, C=4.714866363457395
********* Results ELM-IMA **************
Acc train: 0.9703+/-0.0044
Acc test: 0.9666+/-0.0165
Iterations: 467.1000+/-70.9062
Iterations IMA: 20.0000+/-0.0000
Updates: 7922.6000+/-1502.7182
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9963+/-0.0038
Acc test: 0.8787+/-0.0421
Iterations: 180.3000+/-67.1283
Iterations IMA: 20.0000+/-0.0000
Updates: 1959.2000+/-975.9386
Margin: 0.000000098+/-0.000000164
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6977+/-0.0527
Margin: 0.000001281+/-0.000000117
********* Results SVM **************
Acc train: 0.9887+/-0.0021
Acc test: 0.9824+/-0.0157
Margin: 0.000000+/-0.000000



Experimento com 284 neurônios:
Parameters: p=284, eta=0.1, lambda=0.21387755102040817, C=4.714866363457395
********* Results ELM-IMA **************
Acc train: 0.9691+/-0.0079
Acc test: 0.9648+/-0.0175

In [18]:
sonar_dataset = pd.read_csv('data/sonar/sonar.all-data', names=list(range(0,61)), sep=',')
y = sonar_dataset[60].to_numpy()
X = sonar_dataset.drop([60], axis='columns').to_numpy()
y[np.where(y=='R')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 208 neurônios:
Parameters: p=208, eta=0.1, lambda=7.349591836734694, C=5.963623316594643
********* Results ELM-IMA **************
Acc train: 0.8402+/-0.0451
Acc test: 0.7543+/-0.0890
Iterations: 167.6000+/-24.4139
Iterations IMA: 20.0000+/-0.0000
Updates: 2170.1000+/-238.8315
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9995+/-0.0016
Acc test: 0.6869+/-0.1280
Iterations: 91.1000+/-10.3677
Iterations IMA: 20.0000+/-0.0000
Updates: 593.3000+/-147.7038
Margin: 0.000180437+/-0.000127371
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6583+/-0.1395
Margin: 0.000547988+/-0.000092752
********* Results SVM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.8743+/-0.0921
Margin: 0.000000+/-0.000000



Experimento com 104 neurônios:
Parameters: p=104, eta=0.1, lambda=7.961224489795918, C=5.963623316594643
********* Results ELM-IMA **************
Acc train: 0.8584+/-0.0398
Acc test: 0.7590+/-0.0709
Iter

In [19]:
pima_dataset = pd.read_csv('data/diabetes/diabetes.csv', sep=",", engine='python')
y = pima_dataset['Outcome'].to_numpy()
X = pima_dataset.drop(['Outcome'], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 768 neurônios:
Parameters: p=768, eta=0.1, lambda=4.495306122448979, C=0.4498432668969444
********* Results ELM-IMA **************
Acc train: 0.7652+/-0.0107
Acc test: 0.7513+/-0.0372
Iterations: 253.3000+/-183.3461
Iterations IMA: 20.0000+/-0.0000
Updates: 16191.6000+/-4456.6680
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.8928+/-0.0483
Acc test: 0.5899+/-0.0326
Iterations: 214.4000+/-227.3254
Iterations IMA: 20.0000+/-0.0000
Updates: 10850.4000+/-5676.8889
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5482+/-0.0381
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.7995+/-0.0075
Acc test: 0.7500+/-0.0255
Margin: 0.000000+/-0.000000



Experimento com 384 neurônios:
Parameters: p=384, eta=0.1, lambda=2.660408163265306, C=0.4498432668969444
********* Results ELM-IMA **************
Acc train: 0.7525+/-0.0399
Acc test: 0.7355+/-0

In [20]:
statlog_dataset = pd.read_csv('data/statlog/heart.dat', sep=" ", header=None, engine='python')
y = statlog_dataset[13].to_numpy()
X = statlog_dataset.drop([13], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 270 neurônios:
Parameters: p=270, eta=0.1, lambda=10.0, C=1.8420699693267164
********* Results ELM-IMA **************
Acc train: 0.8321+/-0.0120
Acc test: 0.8037+/-0.0778
Iterations: 128.7000+/-17.6525
Iterations IMA: 20.0000+/-0.0000
Updates: 2579.5000+/-233.8248
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9741+/-0.0289
Acc test: 0.6593+/-0.0873
Iterations: 54.8000+/-9.5058
Iterations IMA: 20.0000+/-0.0000
Updates: 780.1000+/-199.5778
Margin: 0.000000039+/-0.000000069
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5926+/-0.0966
Margin: 0.000000781+/-0.000000361
********* Results SVM **************
Acc train: 0.9037+/-0.0096
Acc test: 0.8333+/-0.0849
Margin: 0.000000+/-0.000000



Experimento com 135 neurônios:
Parameters: p=135, eta=0.1, lambda=9.388367346938775, C=1.8420699693267164
********* Results ELM-IMA **************
Acc train: 0.8399+/-0.0194
Acc test: 0.8296+/-0.0763
Iterations: 94.0

In [21]:
mammo = pd.read_csv('data/mammographic/mammographic_masses.data', sep=",", header=None, engine='python')
mammo = mammo.replace("?", np.nan)
mammo = mammo.dropna()
y = mammo[5].to_numpy()
X = mammo.drop([5], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 830 neurônios:
Parameters: p=830, eta=0.1, lambda=10.0, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.8100+/-0.0128
Acc test: 0.8036+/-0.0573
Iterations: 134.7000+/-54.8198
Iterations IMA: 20.0000+/-0.0000
Updates: 10538.2000+/-2870.9576
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.8466+/-0.0298
Acc test: 0.7892+/-0.0605
Iterations: 101.8000+/-42.7172
Iterations IMA: 20.0000+/-0.0000
Updates: 5842.5000+/-1938.9111
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.8873+/-0.0048
Acc test: 0.7928+/-0.0544
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.8568+/-0.0054
Acc test: 0.8410+/-0.0499
Margin: 0.000000+/-0.000000



Experimento com 415 neurônios:
Parameters: p=415, eta=0.1, lambda=5.922448979591836, C=2.9470517025518097
********* Results ELM-IMA **************
Acc train: 0.7787+/-0.0581
Acc test: 0.7735+/-0.0834
Iterations

In [22]:
haberman = pd.read_csv('data/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 306 neurônios:
Parameters: p=306, eta=0.1, lambda=4.903061224489796, C=49.417133613238384
********* Results ELM-IMA **************
Acc train: 0.6413+/-0.1882
Acc test: 0.6434+/-0.1906
Iterations: 166.1000+/-85.5762
Iterations IMA: 2.9000+/-0.7000
Updates: 10794.2000+/-325.8051
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.7956+/-0.0399
Acc test: 0.6668+/-0.0757
Iterations: 143.4000+/-139.6855
Iterations IMA: 20.0000+/-0.0000
Updates: 2815.9000+/-932.3390
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.8362+/-0.0093
Acc test: 0.6930+/-0.0683
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.7927+/-0.0070
Acc test: 0.7354+/-0.0333
Margin: 0.000000+/-0.000000



Experimento com 153 neurônios:
Parameters: p=153, eta=0.1, lambda=1.233265306122449, C=49.417133613238384
********* Results ELM-IMA **************
Acc train: 0.6893+/-0.1416
Acc test: 0.6918+/-0.1425

In [23]:
transfusion = pd.read_csv('data/transfusion/transfusion.data', sep=",", engine='python')
y = transfusion["whether he/she donated blood in March 2007"].to_numpy()
X = transfusion.drop(["whether he/she donated blood in March 2007"], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 748 neurônios:
Parameters: p=748, eta=0.1, lambda=0.8255102040816327, C=0.1757510624854793
********* Results ELM-IMA **************
Acc train: 0.7246+/-0.0803
Acc test: 0.7300+/-0.0741
Iterations: 376.3000+/-302.4520
Iterations IMA: 2.4000+/-1.0198
Updates: 15534.8000+/-5240.4064
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.7920+/-0.0135
Acc test: 0.7500+/-0.0233
Iterations: 526.4000+/-129.8963
Iterations IMA: 20.0000+/-0.0000
Updates: 59328.2000+/-12433.1756
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 0.8295+/-0.0040
Acc test: 0.7687+/-0.0283
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.7620+/-0.0005
Acc test: 0.7621+/-0.0041
Margin: 0.000000+/-0.000000



Experimento com 374 neurônios:
Parameters: p=374, eta=0.1, lambda=1.8448979591836736, C=0.1757510624854793
********* Results ELM-IMA **************
Acc train: 0.7671+/-0.0042
Acc test: 0.7674+/

In [24]:
australian = pd.read_csv('data/australian_credit/australian.dat', header=None, sep=" ", engine='python')
australian = australian.replace("?", np.nan)
australian = australian.dropna()
y = australian[14].to_numpy()
X = australian.drop([14], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 690 neurônios:
Parameters: p=690, eta=0.1, lambda=6.737959183673469, C=0.054286754393238594
********* Results ELM-IMA **************
Acc train: 0.8324+/-0.0263
Acc test: 0.8275+/-0.0363
Iterations: 222.1000+/-73.6878
Iterations IMA: 20.0000+/-0.0000
Updates: 9478.9000+/-2338.5909
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9602+/-0.0401
Acc test: 0.5014+/-0.0566
Iterations: 172.1000+/-179.2688
Iterations IMA: 20.0000+/-0.0000
Updates: 4518.6000+/-4262.2182
Margin: 0.000000001+/-0.000000001
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.4812+/-0.0434
Margin: 0.000000003+/-0.000000002
********* Results SVM **************
Acc train: 0.8626+/-0.0046
Acc test: 0.8594+/-0.0425
Margin: 0.000000+/-0.000000



Experimento com 345 neurônios:
Parameters: p=345, eta=0.1, lambda=8.368979591836734, C=0.054286754393238594
********* Results ELM-IMA **************
Acc train: 0.8494+/-0.0344
Acc test: 0.8391+/-

In [25]:
breast = pd.read_csv('data/breast/breast.data', header=None, sep=",", engine='python')
breast = breast.replace("?", np.nan)
breast = breast.dropna()
y = breast[10].to_numpy()
X = breast.drop([0, 10], axis='columns').to_numpy()
y[np.where(y==4)] = 1
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 683 neurônios:
Parameters: p=683, eta=0.1, lambda=7.75734693877551, C=0.35564803062231287
********* Results ELM-IMA **************
Acc train: 0.9655+/-0.0065
Acc test: 0.9649+/-0.0118
Iterations: 150.9000+/-50.7059
Iterations IMA: 20.0000+/-0.0000
Updates: 2026.9000+/-555.5843
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9821+/-0.0069
Acc test: 0.8842+/-0.0435
Iterations: 108.8000+/-55.2916
Iterations IMA: 20.0000+/-0.0000
Updates: 1069.5000+/-521.7115
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.8140+/-0.0543
Margin: 0.000000014+/-0.000000002
********* Results SVM **************
Acc train: 0.9702+/-0.0018
Acc test: 0.9663+/-0.0208
Margin: 0.000000+/-0.000000



Experimento com 341 neurônios:
Parameters: p=341, eta=0.1, lambda=4.0875510204081635, C=0.35564803062231287
********* Results ELM-IMA **************
Acc train: 0.9657+/-0.0092
Acc test: 0.9634+/-0.017

In [26]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1
run(X, y)

Experimento com 214 neurônios:
Parameters: p=214, eta=0.1, lambda=8.572857142857142, C=9.540954763499943
********* Results ELM-IMA **************
Acc train: 0.5306+/-0.0955
Acc test: 0.5331+/-0.1035
Iterations: 150.9000+/-27.3768
Iterations IMA: 2.3000+/-0.6403
Updates: 10337.6000+/-171.0913
Margin: 0.000000000+/-0.000000000
********* Results ELM-IMA Beta **************
Acc train: 0.9922+/-0.0234
Acc test: 0.5136+/-0.1263
Iterations: 195.7000+/-119.7598
Iterations IMA: 20.0000+/-0.0000
Updates: 1385.6000+/-778.9999
Margin: 0.000000000+/-0.000000000
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5136+/-0.1280
Margin: 0.000000000+/-0.000000000
********* Results SVM **************
Acc train: 0.8027+/-0.0080
Acc test: 0.7667+/-0.0885
Margin: 0.000000+/-0.000000



Experimento com 107 neurônios:
Parameters: p=107, eta=0.1, lambda=0.21387755102040817, C=9.540954763499943
********* Results ELM-IMA **************
Acc train: 0.6859+/-0.0464
Acc test: 0.6740+/-0.1077

In [27]:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1

In [28]:
spam = pd.read_csv('data/spam/spambase.data', header=None, sep=",", engine='python')
y = spam[57].to_numpy()
X = spam.drop([57], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

KeyboardInterrupt: 